# CS6493 Tutorial 11: LangChain


- Introduction to LangChain

- Create a ChatBot with LangChain + Ollama

- Create a Retrieval Augmented ChatBot with LangChain + Ollama

- Practice: Conversation Retrieval Chain


## Introduction to LangChain

LangChain is a framework for developing applications powered by language models. It enables applications that:

- Are **context-aware**: connect a language model to sources of context (prompt instructions, few shot examples, content to ground its response in, etc.)

- **Reason**: rely on a language model to reason (about how to answer based on provided context, what actions to take, etc.)

This framework consists of several parts.

- **LangChain Libraries:** The Python and JavaScript libraries. Contains interfaces and integrations for a myriad of components, a basic run time for combining these components into chains and agents, and off-the-shelf implementations of chains and agents.
- **LangChain Templates:** A collection of easily deployable reference architectures for a wide variety of tasks.
- **LangServe:** A library for deploying LangChain chains as a REST API.
- **LangSmith:** A developer platform that lets you debug, test, evaluate, and monitor chains built on any LLM framework and seamlessly integrates with LangChain.

Together, these products simplify the entire application lifecycle:

- **Develop:** 88 Write your applications in LangChain/LangChain.js. Hit the ground running using Templates for reference.
- **Productionize:** Use LangSmith to inspect, test and monitor your chains, so that you can constantly improve and deploy with confidence.
- **Deploy:** Turn any chain into an API with LangServe.

LangChain Libraries

The main value props of the LangChain packages are:

- **Components:** composable tools and integrations for working with language models. Components are modular and easy-to-use, whether you are using the rest of the LangChain framework or not
- **Off-the-shelf chains:** built-in assemblages of components for accomplishing higher-level tasks

Off-the-shelf chains make it easy to get started. Components make it easy to customize existing chains and build new ones.

The LangChain libraries themselves are made up of several different packages.

- **langchain-core:** Base abstractions and LangChain Expression Language.
- **langchain-community:** Third party integrations.
- **langchain:** Chains, agents, and retrieval strategies that make up an application's cognitive architecture.

## Create a ChatBot with LangChain + Ollama

LangChain enables building application that connect external sources of data and computation to LLMs. In this quickstart, we will walk through a few different ways of doing that. We will start with a simple LLM chain, which just relies on information in the prompt template to respond. Next, we will build a retrieval chain, which fetches data from a separate database and passes that into the prompt template. We will then add in chat history, to create a conversation retrieval chain. This allows you to interact in a chat manner with this LLM, so it remembers previous questions.

Install

In [ ]:
!pip install langchain

In [ ]:
!pip install langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 44.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


We can use models available via API, like OpenAI, and local open source models, using integrations like Ollama. Here we choose to use Ollama.

First, let's install Ollama and start Ollama service

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!setsid nohup ollama serve > run_serve.txt 2>&1 & # we must use this command to make sure the ollama serve is running

In [ ]:
# check models we can use
!ollama list

NAME    ID    SIZE    MODIFIED 


We should firstly choose a model as the backbone model for our application. To reduce the cost of computation resources, here we just choose gemma-2b model as our backbone model.

In [ ]:
!setsid ollama run gemma:2b > run_gemma_2b.txt 2>&1 &

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="gemma:2b")



<ipython-input-10-e6f32ec21979>:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma:2b")


Once you've installed and initialized the LLM of your choice, we can try using it! Let's ask it what LangSmith is - this is something that wasn't present in the training data so it shouldn't have a very good response.

In [ ]:
llm.invoke("how can langsmith help with testing?")

'**Langsmith can help with testing in several ways:**\n\n**1. Test data generation:**\n* Generate realistic and diverse test data for various testing scenarios.\n* Create mock data for external APIs, databases, and other components.\n* Generate test cases based on specific requirements and functionalities.\n\n**2. Code coverage analysis:**\n* Identify which parts of the code are being tested through test case execution.\n* Highlight areas where additional testing may be needed.\n* Analyze code coverage over multiple iterations and iterations.\n\n**3. Error detection and reporting:**\n* Identify and report syntax, logic, and runtime errors.\n* Detect potential defects and areas for improvement.\n* Generate detailed test reports for easy analysis.\n\n**4. Test case management:**\n* Organize and manage test cases in a centralized repository.\n* Track test execution status and results.\n* Facilitate communication and collaboration among testers.\n\n**5. Automated testing:**\n* Generate aut

We can also guide its response with a prompt template. Prompt templates convert raw user input to better input to the LLM.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

We can now combine these into a simple LLM chain:

In [ ]:
chain = prompt | llm

We can now invoke it and ask the same question. It still won't know the answer, but it should respond in a more proper tone for a technical writer!

In [ ]:
chain.invoke({"input": "how can langsmith help with testing?"})

"**Langsmith can help with testing in a few ways:**\n\n**1. Identifying potential testing gaps:**\n\n* Langsmith can analyze the requirements, architecture, and design of your software system.\n* By identifying potential testing gaps and areas that need to be tested, you can prioritize your testing efforts and focus on areas that will have the biggest impact on the quality of your product.\n\n**2. Generating realistic test cases:**\n\n* Langsmith can automatically generate test cases from the system requirements and other documentation.\n* This can save you time and effort, allowing you to focus on writing more complex and challenging test cases.\n\n**3. Identifying potential edge cases and corner cases:**\n\n* Langsmith can simulate different scenarios and edge cases that your users might encounter.\n* This can help you identify potential bugs and defects that you might otherwise miss during manual testing.\n\n**4. Creating detailed and clear test reports:**\n\n* Langsmith can automat

The output of a ChatModel (and therefore, of this chain) is a message. However, it's often much more convenient to work with strings. Let's add a simple output parser to convert the chat message to a string.

In [15]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

We can now add this to the previous chain:

In [16]:
chain = prompt | llm | output_parser

We can now invoke it and ask the same question. The answer will now be a string (rather than a ChatMessage).

In [17]:
chain.invoke({"input": "how can langsmith help with testing?"})

'**Langsmith can help with testing in several ways:**\n\n**1. Identifying potential testing issues:**\n\n* **Natural language descriptions:** You can define the testing steps and scenarios using natural language, allowing users to easily express their testing requirements.\n* **Automating tests:** Langsmith can extract the testing steps from natural language descriptions and automatically generate automated tests, eliminating the need for manual coding.\n\n**2. Providing clear and concise documentation:**\n\n* **Structured test cases:** You can define test cases in a structured format, including prerequisites, steps, and expected results. This makes it easier for developers and testers to understand and execute the tests.\n* **Improved documentation:** Langsmith can generate comprehensive documentation from your test cases, including clear and concise descriptions of the testing process, resulting in better knowledge transfer and collaboration.\n\n**3. Enhancing communication between s

## Retrieval Chain

To properly answer the original question ("how can langsmith help with testing?"), we need to provide additional context to the LLM. We can do this via retrieval. Retrieval is useful when you have too much data to pass to the LLM directly. You can then use a retriever to fetch only the most relevant pieces and pass those in.

In this process, we will look up relevant documents from a Retriever and then pass them into the prompt. A Retriever can be backed by anything - a SQL table, the internet, etc - but in this instance we will populate a vector store and use that as a retriever. For more information on vectorstores, see this documentation.

First, we need to load the data that we want to index. To do this, we will use the WebBaseLoader. This requires installing BeautifulSoup:

In [18]:
!pip install beautifulsoup4

After that, we can import and use WebBaseLoader.

In [19]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

docs = loader.load()

Next, we need to index it into a vectorstore. This requires a few components, namely an embedding model and a vectorstore.

For embedding models, we once again provide examples for accessing via API or by running local models.

In [20]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="gemma:2b")

<ipython-input-20-86ccdb9bb266>:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embeddings = OllamaEmbeddings(model="gemma:2b")


Now, we can use this embedding model to ingest documents into a vectorstore. We will use a simple local vectorstore, FAISS, for simplicity's sake.

First we need to install the required packages for that:

In [21]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 63.9 MB/s eta 0:00:00


Then we can build our index:

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

Now that we have this data indexed in a vectorstore, we will create a retrieval chain. This chain will take an incoming question, look up relevant documents, then pass those documents along with the original question into an LLM and ask it to answer the original question.

First, let's set up the chain that takes a question and the retrieved documents and generates an answer.

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

If we wanted to, we could run this ourselves by passing in documents directly:

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

'The context does not provide any information about how langsmith can help with testing, so I cannot answer this question from the provided context.'

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

We can now invoke this chain. This returns a dictionary - the response from the LLM is in the answer key

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

Langsmith can help with testing by providing a variety of tools and features that make it easier to identify and root-cause issues in your LLM applications. These tools include:

- Prototyping: Langsmith allows you to quickly experiment between prompts, model types, retrieval strategy and other parameters.
- Debugging: When developing new LLM applications, you can create datasets to run tests on your applications and use these datasets to run custom evaluations.
- Comparison View: You can view results for different configurations on the same datapoints side-by-side to track and diagnose regressions in test scores across multiple revisions of your application.
- Playground: You can quickly test out different prompts and models in the playground environment.
- Beta Testing: You can collect more data on how your LLM applications are performing in real-world scenarios in the beta testing phase.
- Capturing Feedback: When launching your application to an initial set of users, you can gather

# Practice: Conversation Retrieval Chain

The chain we've created so far can only answer single questions. One of the main types of LLM applications that people are building are chat bots. So how do we turn this chain into one that can answer follow up questions?

We can still use the create_retrieval_chain function, but we need to change two things:

-The retrieval method should now not just work on the most recent input, but rather should take the whole history into account.

-The final LLM chain should likewise take the whole history into account

Updating Retrieval

In order to update retrieval, we will create a new chain. This chain will take in the most recent input (input) and the conversation history (chat_history) and use an LLM to generate a search query.

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

We can test this out by passing in an instance where the user asks a follow-up question.

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(id='da4515f8-6d3b-4edb-b859-79af91e2eb3b', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.', 'language': 'en'}, page_content="Beta Testing‚Äã\nBeta testing allows developers to collect more data on how their LLM applications are performing in real-world scenarios. In this phase, it‚Äôs important to develop an understanding for the types of inputs the app is performing well or poorly on and how exactly it‚Äôs breaking down in those cases. Both feedback collection and run annotation are critical f

You should see that this returns documents about testing in LangSmith. This is because the LLM generated a new query, combining the chat history with the follow-up question.

Now that we have this new retriever, we can create a new chain to continue the conversation with these retrieved documents in mind.

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

We can now test this out end-to-end:

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Yes!', additional_kwargs={}, response_metadata={})],
 'input': 'Tell me how',
 'context': [Document(id='da4515f8-6d3b-4edb-b859-79af91e2eb3b', metadata={'source': 'https://docs.smith.langchain.com/user_guide', 'title': 'LangSmith User Guide | \uf8ffü¶úÔ∏è\uf8ffüõ†Ô∏è LangSmith', 'description': 'LangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we‚Äôll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they‚Äôre just starting their journey.', 'language': 'en'}, page_content="Beta Testing‚Äã\nBeta testing allows developers to collect more data on how their LLM applications are performing in real-world sce